# Image Quality
- Brisque - https://pypi.org/project/brisque/
- image-quality 1.2.7 - https://pypi.org/project/image-quality/
- NIMA - https://github.com/yunxiaoshi/Neural-IMage-Assessment

In [2]:
import time
import imquality.brisque
import PIL.Image
from brisque import BRISQUE
from skimage import io
import torchvision.transforms as transforms
import torch
import piq
from skimage.io import imread
import warnings
import os
import numpy as np
import skimage
import cv2

In [3]:
path1 = '/home/lukas/Bakalářka/images/Ples/fotokoutek/foto0001.jpg'
img1 = io.imread(path1)
path2 = '/home/lukas/Bakalářka/images/Ples/fotokoutek/foto0002.jpg'
img2 = io.imread(path2)
path3 = '/home/lukas/Bakalářka/images/Ples/fotokoutek/foto0004.jpg'
img3 = io.imread(path3)

In [95]:
### BRISQUE - technical quality only
time1 = time.perf_counter()
obj = BRISQUE(url=False)
print("SCORE:",obj.score(img3)/10)
time2 = time.perf_counter()
print(f"TIME - {time2-time1:0.2f} s")

SCORE: 2.7007605797649576
TIME - 0.80 s


In [96]:
### imquality - technical quality only
#  - implementation of brisque 
#  - very slow
time1 = time.perf_counter()
print("SCORE:",imquality.brisque.score(PIL.Image.open(path3))/10)
time2 = time.perf_counter()
print(f"TIME - {time2-time1:0.2f} s")

SCORE: 2.587213443716152
TIME - 5.87 s


In [97]:
### NIMA - aesthetic and technical quality depending on training database
from main import prepare_model
time1 = time.perf_counter()
model,device = prepare_model("epoch-82.pth") # path to model
test_transform = transforms.Compose([
        transforms.Resize(256),
        transforms.RandomCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])
imt = test_transform(PIL.Image.open(path3))
imt = imt.unsqueeze(dim=0)
imt = imt.to(device)
with torch.no_grad():
    out_f, out_class = model(imt)
out_class = out_class.view(10, 1)
mean, std = 0.0, 0.0
for j, e in enumerate(out_class, 1):
    mean += j * e
for k, e in enumerate(out_class, 1):
    std += e * (k - mean) ** 2
std = std ** 0.5
print(f"MEAN SCORE: {float(mean):0.2f} STD: {float(std):0.2f}")
time2 = time.perf_counter()
print(f"TIME - {time2-time1:0.2f} s")

MEAN SCORE: 6.29 STD: 1.34
TIME - 2.18 s


# Image Similarity
- PIQ - https://github.com/photosynthesis-team/piq --> different tools and metrics to find similar images
- Skimage - https://scikit-image.org/docs/stable/api/skimage.metrics.html#skimage.metrics.structural_similarity
- SIFT - https://github.com/adumrewal/SIFTImageSimilarity

In [98]:
# PIQ DISTS function
# - problem with different resolutions --> after resizing too much information lost and inaccurate
warnings.filterwarnings('ignore')

time1 = time.perf_counter()
x = torch.tensor(imread('~/Bakalářka/images/Ples/fotokoutek/foto0001.jpg')).permute(2, 0, 1)[None, ...] / 255.
y = torch.tensor(imread('~/Bakalářka/images/Ples/fotokoutek/foto0002.jpg')).permute(2, 0, 1)[None, ...] / 255.
z = torch.tensor(imread('~/Bakalářka/images/Ples/fotokoutek/foto0005.jpg')).permute(2, 0, 1)[None, ...] / 255.

dists_loss_1 = piq.DISTS(reduction='none')(x, y)
dists_loss_2 = piq.DISTS(reduction='none')(x, z)
print("Smaller number means more similar images")
print(f"DISTS: {dists_loss_1.item():0.4f}")
print(f"DISTS: {dists_loss_2.item():0.4f}")
time2 = time.perf_counter()
print(f"TIME - {time2-time1:0.2f} s")

Smaller number means more similar images
DISTS: 0.0581
DISTS: 0.2703
TIME - 5.33 s


In [99]:
### Skimage
#   - outdated implementation with square complexity 
#   - high memory and cpu usage for mediocre results 
#   - faster than PIQ DISTS function

pth_images = '/home/lukas/Bakalářka/images/Ples/fotokoutek'
SIM_LIM = 5  # limiting the number of images for fast example
SIM_THRESHOLD = 0.7  # threshold for similarity

def imread_reshape(path_images):
    min_w = float('inf')
    min_h = float('inf')
    num = 0
    i = 0
    lst = os.listdir(path_images)
    lst.sort()
    for image in lst:
        if i >= SIM_LIM:
            break
        i+=1
        temp = io.imread(os.path.join(path_images, image))
        if temp is not None:
            num += 1
            if min_w > temp.shape[0]:
                min_w = temp.shape[0]
            if min_h > temp.shape[1]:
                min_h = temp.shape[1]
    shape = [min_w, min_h]

    print('FOUND shape:', shape)
    print('Preparing images...')

    images = None
    i = 0
    for image in lst:
        if i >= SIM_LIM:
            break
        i+=1
        temp = torch.tensor(io.imread(os.path.join(path_images, image)))/255.
        if temp is not None:
            temp = skimage.transform.resize_local_mean(temp, output_shape=shape)
            temp = skimage.color.rgb2gray(temp)
            temp = np.expand_dims(temp, 2)
            temp = np.rollaxis(temp, 2, 0).astype('f2')
            if images is None:
                images = temp
            else:
                images = np.append(images,temp,0).astype('f2')
    print('Images prepared')
    return images, num, lst

time1 = time.perf_counter()
ic, img_num, img_lst = imread_reshape(pth_images)
time2 = time.perf_counter()

print(f"TIME to process images - {time2-time1:0.2f} s")

sim_list = [[0 for i in range(img_num)] for j in range(img_num)]


time1 = time.perf_counter()
for i in range(img_num):
    for j in range(img_num):
        sim_list[i][j] = skimage.metrics.structural_similarity(ic[:][:][:][i], ic[:][:][:][j], win_size=3, data_range= 2)
        print(' {0}/{1}'.format(i*img_num+(j+1),img_num*img_num),end='\r')
time2 = time.perf_counter()
print(f"TIME to get all similar - {time2-time1:0.2f} s")

for i in range(len(sim_list[0])):
    most_similar = 0
    most_idx = 0
    for j in range(len(sim_list[0])):
        if i == j:
            continue
        if sim_list[i][j] > most_similar:
            most_similar =  sim_list[i][j]
            most_idx = j
    if most_similar > SIM_THRESHOLD:
        print(img_lst[i],' most similar img is',img_lst[most_idx], 'with the score of', most_similar)
    else:
        print(img_lst[i],' doesnt have similar image in set')

FOUND shape: [799, 799]
Preparing images...
Images prepared
TIME to process images - 0.62 s
TIME to get all similar - 1.08 s
foto0001.jpg  most similar img is foto0002.jpg with the score of 0.7743849043067754
foto0002.jpg  most similar img is foto0001.jpg with the score of 0.7743849043067754
foto0003.jpg  most similar img is foto0001.jpg with the score of 0.7407976082617311
foto0004.jpg  doesnt have similar image in set
foto0005.jpg  doesnt have similar image in set


In [1]:
from main import calculate_score,calculate_matches,compute_SIFT,image_resize,prepare_img_list

pth = '/home/lukas/Bakalářka/images/Ples/fotokoutek'
nbrs = 5
SIM_LIM = 20
lst,num = prepare_img_list(pth)
if num > SIM_LIM:
    num = SIM_LIM

features={} # keypoints and descriptors
sim_list = [] # list to store results
time1 = time.perf_counter()
for i in range(num):
    img = image_resize(cv2.imread(os.path.join(pth,lst[i])))
    keypoints, descriptors = compute_SIFT(img)
    features[i] = (keypoints,descriptors)
for i in range(num):
    keypoints_i, descriptors_i = features[i]
    for j in range(max(0,i-nbrs),min(num,i+nbrs+1)):
        if i>=j:
            continue
        keypoints_j, descriptors_j = features[j]
        matches = calculate_matches(descriptors_i, descriptors_j)
        score = calculate_score(len(matches), len(keypoints_i), len(keypoints_j))
        print("IMAGES:",lst[i], lst[j],"SIMILARITY SCORE:",score)
time2 = time.perf_counter()
print(f"TIME - {time2-time1:0.2f} s")


NameError: name 'os' is not defined

# Image Content
- VGG16 - https://towardsdatascience.com/how-to-use-a-pre-trained-model-vgg-for-image-classification-8dd7c4a4a517
- Bag of words approach from similarity search results

In [37]:
from keras.applications.vgg16 import VGG16
from keras import preprocessing
from main import prepare_img_list,pred_result

class_model = VGG16(weights='imagenet')
SIM_LIM = 1

img_path = "/home/lukas/Bakalářka/images/Ples/fotokoutek"
content_path = "image_content_fotokoutek.json"
lst,_ = prepare_img_list(img_path)

time1 = time.perf_counter()
for i, p in enumerate(lst):
    if i >= SIM_LIM:
        break
    img = preprocessing.image.load_img(os.path.join(img_path,p),color_mode='rgb', target_size=(224, 224))
    res = pred_result(img)
    print(res)
time2 = time.perf_counter()
print(f"TIME - {time2-time1:0.2f} s")

ImportError: cannot import name 'pred_result' from 'main' (/home/lukas/Bakalářka/main.py)